In [1]:
# import packages
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV

In [2]:
# load dataset
fifa = pd.read_csv("FIFA19data.csv", sep=r'\s*,\s*', engine='python')
fifa.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,5.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,5.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,5.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,4.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,4.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [3]:
# drop variables
fifa = fifa.drop('ID', 1)
fifa = fifa.drop('Name', 1)
fifa = fifa.drop('Nationality', 1)
fifa = fifa.drop('Club', 1)
fifa = fifa.drop('Value', 1)
fifa = fifa.drop('Wage', 1)
fifa = fifa.drop('Body Type', 1)
fifa = fifa.drop('Potential', 1)

In [4]:
# create dummy variables
for col in fifa.columns:
    fifa[col].fillna(value=fifa[col].mode()[0], inplace=True)

factors = ['International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Position', 'Contract Valid Until']

for var in factors:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(fifa[var], prefix=var)
    fifa = pd.concat([fifa,cat_list], axis = 1)
    fifa = fifa.drop(var, 1)

In [21]:
# create X and Y dataset
X = fifa.copy()
X = X.drop('Overall', 1)
Y = fifa.copy()
Y = Y['Overall']
print('number of variables:', X.shape[1])

number of variables: 122


In [6]:
# randomly split data into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.9, random_state=31)

# Basic Linear Model

In [7]:
# establish simple linear regression model 
lm1 = LinearRegression()
lm1.fit(X_train, y_train)
lm1_predictions = lm1.predict(X_test)
lm1_r2 = r2_score(y_test,lm1_predictions)
w = lm1.coef_
print('number of feature used:', w.shape[0])
print('R-square:', lm1_r2)

number of feature used: 122
R-square: 0.8904970737556855


# Cross Validation

In [8]:
# use 5-fold cross validation to fit the linear regression model
cv_predictions = cross_val_predict(lm1, X_test, y_test, cv=5)
cv_r2 = r2_score(y_test,cv_predictions)
print('R-square:', cv_r2)

R-square: 0.895506109826533


# Lasso Regression

## Lasso Regression with default alpha(1)

In [9]:
# use lasso regression to select significant variables and establish model
lasso = Lasso()
lasso.fit(X_train,y_train)
lasso1_predictions = lasso.predict(X_test)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)
r2_lasso1 = r2_score(y_test, lasso1_predictions)

In [10]:
print("training score:", train_score)
print("test score: ", test_score)
print("number of features used: ", coeff_used)
print("test r2 score: ", r2_lasso1)

training score: 0.8584100702105251
test score:  0.8508221432029555
number of features used:  23
test r2 score:  0.8508221432029555


## Lasso Regression with best alpha

We used two methods to test the best alpha, one is lassoCV, and the other one is GridSearchCV.

### Approach 1: LassoCV

LassoCV is the function which use cross validation to find the best model with best alpha. 

In [11]:
# find the best alpha
lassocv = LassoCV(alphas = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20], cv=5, random_state=31)
lassocv.fit(X_train,y_train)
best_alpha = lassocv.alpha_
print('alpha:', best_alpha)

# calculate train score, test score and r-square
lasso = Lasso(alpha = best_alpha)
lasso.fit(X_train, y_train)
train_score = lasso.score(X_train, y_train)
test_score=lasso.score(X_test,y_test)
lasso_predictions = lasso.predict(X_test)
r2_lasso = r2_score(y_test, lasso_predictions)
print('train score:', train_score)
print('test score:', test_score)
print('R-square:', r2_lasso)

C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 198.31850021038827, tolerance: 7.077029670329669
  tol, rng, random, positive)
C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 198.28051467929708, tolerance: 7.077029670329669
  tol, rng, random, positive)
C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3295.289835660465, tolerance: 7.077029670329669
  tol, rng, random, positive)
C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You mi

alpha: 0.01
train score: 0.900304099950968
test score: 0.8898243448923757
R-square: 0.8898243448923757


In [12]:
# print number of feature used
coeff_used = np.sum(lasso.coef_!=0)
print('number of feature used:', coeff_used)

number of feature used: 59


### Approach 2: GridSearchCV

GridSearchCV uses grid search and cross validation technique to test all possible alpha value to find the best model.

In [13]:
lasso = Lasso()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
lasso_regressor = GridSearchCV(lasso, parameters, cv = 5)
lasso_regressor.fit(X_train, y_train)

C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3372.9529011550253, tolerance: 7.077029670329669
  positive)
C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3374.0470912585624, tolerance: 7.259854670329671
  positive)
C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3263.4799344132684, tolerance: 7.131133997252747
  positive)
C:\Users\whn97\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                   5, 10, 20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [14]:
# find the best alpha
lasso_regressor.best_params_

{'alpha': 0.01}

In [15]:
# print number of feature used
coeff_used = np.sum(lasso_regressor.best_estimator_.coef_!=0)
print('number of feature used:', coeff_used)

number of feature used: 59


In [16]:
# print train score and test score
train_score = lasso_regressor.score(X_train,y_train)
test_score = lasso_regressor.score(X_test, y_test)
print('train score:', train_score)
print('test score:', test_score)

train score: 0.900304099950968
test score: 0.8898243448923757


In [17]:
# print r-square
lasso2_predictions = lasso_regressor.predict(X_test)
r2_lasso2 = r2_score(y_test, lasso2_predictions)
print('R-square:', r2_lasso2)

R-square: 0.8898243448923757


As we can see, the two approaches got the same results. 
Best alpha = 0.01; 
train score = 0.9003; 
test score = 0.8898; 
R-square = 0.8898; 

# AIC BIC

In [18]:
# define the function for aic, aicc and bic
def AIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + 2*coeff_used

def AICc(y_true, y_hat, coeff_used):
    n = len(y_hat)
    return AIC(y_true, y_hat, coeff_used) + 2*coeff_used*(coeff_used+1)/(n-coeff_used-1)

def BIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + np.log(n)*coeff_used

In [19]:
#aic, aicc and bic of simple linear model
aic_lm1 = AIC(y_test, lm1_predictions, (len(X_test.columns)+1))
print(aic_lm1)
aicc_lm1 = AICc(y_test, lm1_predictions, (len(X_test.columns)+1))
print(aicc_lm1)
bic_lm1 = BIC(y_test, lm1_predictions, (len(X_test.columns)+1))
print(bic_lm1)

27300.326786055277
27302.20245475109
28247.948750890464


In [20]:
#aic, aicc and bic of lasso model
aic_lasso2 = AIC(y_test, lasso2_predictions, (coeff_used+1))
print(aic_lasso2)
aicc_lasso2 = AICc(y_test, lasso2_predictions, (coeff_used+1))
print(aicc_lasso2)
bic_lasso2 = BIC(y_test, lasso2_predictions, (coeff_used+1))
print(bic_lasso2)

27274.691985757767
27275.140350329617
27736.94660275054
